Name: Ayusha Kakkad

Student ID: 22210321

## Task1 : Generate Top View

For TopView Generation following steps were followed which will be explained in detail during the code implementation:

1) Loading of Images in the Dataset (Mono-Left, Mono-Right, Mono-Rear,Stereo-Centre,Stereo-Right,Stereo-Left)

2) Convert the Grey Images in Dataset to RGB

3) Demosaic all the Images

4) For all FishEye Calibrated Images Undistortion is performed

5) WrapPerspective Transform Function is applied on Mono-Left,Stereo Centre,Mono-Right and Mono-Rear

6) Images are Rotated to generate an appropriate Top-View.

7) Top-View is generated by Stitching the 4 set of images rotated in above step

### Package Installations

In [ ]:
# Installing the colour_demosaicing function required for demosaic function
!pip install colour_demosaicing

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import random
import re
from PIL import Image
from colour_demosaicing import demosaicing_CFA_Bayer_bilinear as demosaic
%matplotlib inline

### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### RGB + Demosiac Function

Demosaic Function reconstructs a grey image provided in the dataset to a full-resolution color image from the sampled data acquired by a digital camera model. The digital camera model applies a color filter array to a single sensor.

If the image is part of Stereo images then images are only demosaiced. For mono images undistortion is also performed.

In [ ]:
BAYER_STEREO = 'gbrg'
BAYER_MONO = 'rggb'

#image_path = provides path to images stored in database
def load_image(image_path, model=None, debayer=True):

    if model:
        camera = model.camera
    else:
        camera = re.search('(stereo|mono_(left|right|rear))', image_path).group(0)
#if stereo demosaic
    if camera == 'stereo':
        pattern = BAYER_STEREO
    else:
        pattern = BAYER_MONO

    img = Image.open(image_path)
    if debayer:
        img = demosaic(img, pattern)
#if mono undistort which is implemented in future code
    if camera=='mono':
        img = model.undistort(img)

    return np.array(img).astype(np.uint8)

### Load Images

Google Drive is Mounted and the Dataset consisting of Mono-Right,Mono Left,Mono-Rear, Stereo-Centre,Stereo-Right,Stereo-Left folders are loaded which consist of 40 images each.

Originally there were 49 images in the stereo folders but for implementation the last 9 images have been removed.

With the help of matplotlib.pyplot as plt I have plot all images are plot below. enumerate is used for providing the count of images.str(img+1) provides the number of images as they print.

In [ ]:
def load_images_from_folder(input_path):
    filenames = [input_path + filename for filename in os.listdir(input_path)]
    images = []
    for filename in filenames:
            image = load_image(filename)
            images.append(image)
    return images

####Mono Left Images

In [ ]:
mono_left_images = load_images_from_folder('/content/drive/MyDrive/sample_small/mono_left/')
for img, image in enumerate(mono_left_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Left Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

####Mono Right Images

In [ ]:
mono_right_images = load_images_from_folder('/content/drive/MyDrive/sample_small/mono_right/')
for img, image in enumerate(mono_right_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Right Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

#### Mono Rear Images

In [ ]:
mono_rear_images= load_images_from_folder('/content/drive/MyDrive/sample_small/mono_rear/')
for img, image in enumerate(mono_rear_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Rear Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

#### Stereo Centre Images

In [ ]:
stereo_centre_images = load_images_from_folder('/content/drive/MyDrive/sample_small/stereo/centre/')
for img, image in enumerate(stereo_centre_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Stereo Centre Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

#### Stereo Left Images

In [ ]:
stereo_left_images =load_images_from_folder('/content/drive/MyDrive/sample_small/stereo/left/')
for img, image in enumerate(stereo_left_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Stereo Left Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

####Stereo Right Images

In [ ]:
stereo_right_images=load_images_from_folder('/content/drive/MyDrive/sample_small/stereo/right/')
for img, image in enumerate(stereo_right_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Stereo Right Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

####Improving Colour Shade of Stereo Centre Image

Stereo Images have a blusish shade of roads post demosaicing the images. For generating accurate Top View, the colour shade of Stereo Centre(Front) Images are improved by Histogram Matching.

Histogram matching can be used as a lightweight normalisation for image processing, such as feature matching,where the images have been taken in different conditions. The stereo centre images are taken and matched with mono rear images colour shade and histogram.

In [ ]:
from skimage.exposure import cumulative_distribution
from skimage.color import rgb2gray

def hist_matching(c, c_t, im):
    b = np.interp(c, c_t, np.arange(256))   # find closest matches to b_t
    pix_repl = {i:b[i] for i in range(256)} # dictionary to replace the pixels
    mp = np.arange(0,256)
    for (k, v) in pix_repl.items():
        mp[k] = v
    s = im.shape
    im = np.reshape(mp[im.ravel()], im.shape)
    im = np.reshape(im, s)
    return im

def cdf(im):
    c, b = cumulative_distribution(im)
    for i in range(b[0]):
        c = np.insert(c, 0, 0)
    for i in range(b[-1]+1, 256):
        c = np.append(c, 1)
    return c

# Load stereo and mono_rear images
img2 = mono_rear_images[0]
for i in range(len(stereo_centre_images)):
    img1 = stereo_centre_images[i]

    # Match colors
    im1 = np.zeros(img1.shape).astype(np.uint8)
    for j in range(3):
        c = cdf(img1[...,j])
        c_t = cdf(img2[...,j])
        im1[...,j] = hist_matching(c, c_t, img1[...,j])
    stereo_centre_images[i] = im1

#Plot Images
for img, image in enumerate(stereo_centre_images):
    plt.figure(figsize=(5,5))
    plt.title(str("Stereo Centre Improvised Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

###Undistorting Images

For Undistorting the fish eye calibrated mono images, first we obtain the model name (stereo left|centre|right or mono left|right|rear),next we obtain the image directory with model name + from  models folder which ends with _distortion_lut.bin file. After getting both,the undistortion function is applied.

There is a check for "Incorrect image size for camera model" and whether the image being undistorted is an multi-channel image.

Image distortion is when the straight lines of an image appear to be deformed or curved unnaturally. Distortion can significantly disrupt the image's quality. Undistorting the image will result in better quality of the image and straight lines in the image.

In [ ]:
#get image
def load_lut(models_dir, images_dir):
  model_name = __get_model_name(images_dir)
  lut_path = os.path.join(models_dir, model_name + '_distortion_lut.bin')
  lut = np.fromfile(lut_path, np.double)
  lut = lut.reshape([2, lut.size // 2])
  bilinear_lut = lut.transpose()
  return bilinear_lut

#get model name
def __get_model_name(images_dir):
  camera = re.search('(stereo|mono_(left|right|rear))', images_dir).group(0)
  if camera == 'stereo':
    camera_sensor = re.search('(left|centre|right)', images_dir).group(0)
    if camera_sensor == 'left':
      return 'stereo_wide_left'
    elif camera_sensor == 'right':
      return 'stereo_wide_right'
    elif camera_sensor == 'centre':
      return 'stereo_narrow_left'
    else:
      raise RuntimeError('Unknown camera model for given directory: ' + images_dir)
  else:
    return camera

In [ ]:
models_dir = ('/content/drive/MyDrive/sample_small/models')
images_dir = ('/content/drive/MyDrive/sample_small/mono_left')
__get_model_name(images_dir)
load_lut(models_dir,images_dir)

In [ ]:
from scipy.ndimage import map_coordinates

def undistort(image):
  bilinear_lut = load_lut(models_dir,images_dir)
  if image.shape[0] * image.shape[1] != bilinear_lut.shape[0]:
    raise ValueError('Incorrect image size for camera model')
  lut = bilinear_lut[:, 1::-1].T.reshape((2, image.shape[0], image.shape[1]))
  if len(image.shape) == 1:
      raise ValueError('Undistortion function only works with multi-channel images')
  undistorted = np.rollaxis(np.array([map_coordinates(image[:, :, channel], lut, order=1)
      for channel in range(0, image.shape[2])]), 0, 3)
  return undistorted.astype(image.dtype)

####Mono Left Images

In [ ]:
undistort_mono_left = []
for img in mono_left_images:
  undistort_img = undistort(img)
  undistort_mono_left.append(undistort_img)

for img, image in enumerate(undistort_mono_left):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Left Undistorted Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

####Mono Right Images

In [ ]:
undistort_mono_right = []
for img in mono_right_images:
  undistort_img = undistort(img)
  undistort_mono_right.append(undistort_img)

for img, image in enumerate(undistort_mono_right):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Right Undistorted Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

#### Mono Rear Images

In [ ]:
undistort_mono_rear = []
for img in mono_rear_images:
  undistort_img = undistort(img)
  undistort_mono_rear.append(undistort_img)

for img, image in enumerate(undistort_mono_rear):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Rear Undistorted Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

###WarpPerspective Function

An array of four points is sent to the "order points" function, which outputs the points in the following order: top-left, top-right, bottom-right, top-left.The function achieves this by finding the points with the minimum and maximum sums and differences of their x and y coordinates.

To obtain the ordered points of the rectangle in the image, the "four point transform" function first invokes the "order points" function. It then applies Euclidean distance between the points on the opposing sides of the rectangle to determine the maximum width and height of the resulting transformed image. The transformed coordinates of the rectangle's four corners are represented by a destination array of points.

The function then calculates a perspective transformation matrix (M) using the "cv2.getPerspectiveTransform" function, which maps the original image to the transformed image.

Finally, the "cv2.warpPerspective" function is called to apply the transformation to the original image using the perspective transformation matrix.

Seperate Points are set for each mono and stereo centre images.

In [ ]:
def order_points(pts):

	rect = np.zeros((4, 2), dtype = "float32")

	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [ ]:
def four_point_transform(image, pts):

	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")

	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	return warped


####Mono Left Images

Mono-Left Images are rotated to 90 degrees Anticlockwise to present Top-View Generation.

In [ ]:
wrap_mono_left = []
pts = np.array(eval(("[[10,650], [1023,450], [1023,800], [100,800]]")))
for img in undistort_mono_left:
  wrap_image = four_point_transform(img,pts)
  rotated_img_left = cv2.rotate(wrap_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
  wrap_mono_left.append(rotated_img_left)

for img, image in enumerate(wrap_mono_left):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Left Wrap Perspective Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

#### Mono Right Images

Mono-Right Images are rotated to 90 degrees clockwise to present Top-View Generation.

In [ ]:
wrap_mono_right = []
pts = np.array(eval(("[[10,650], [1023,450], [1023,800], [100,800]]")))
for img in undistort_mono_right:
  wrap_image = four_point_transform(img,pts)

# Rotate the image
  rotated_img_right = cv2.rotate(wrap_image, cv2.ROTATE_90_CLOCKWISE)
  wrap_mono_right.append(rotated_img_right)

for img, image in enumerate(wrap_mono_right):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Right Wrap Perspective Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

####Mono Rear Images

Images are resized so that images can be placed properly in order on the blank image below during stitching. Mono-Rear Images are rotated to 180 degrees to present Top-View Generation.

In [ ]:
wrap_mono_rear = []
pts = np.array(eval(("[[10,650], [1023,450], [1023,800], [100,800]]")))
for img in undistort_mono_rear:
  wrap_image = four_point_transform(img,pts)
  #resize images
  wrap_image = cv2.resize(wrap_image, (1015, 200))
  rotated_img_rear = cv2.rotate(wrap_image, cv2.ROTATE_180)
  wrap_mono_rear.append(rotated_img_rear)

for img, image in enumerate(wrap_mono_rear):
    plt.figure(figsize=(5,5))
    plt.title(str("Mono Rear Wrap Perspective Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

#### Stereo Centre Images

Images are resized so that images can be placed properly in order on the blank image below during stitching.

In [ ]:
wrap_stereo_centre = []
pts = np.array(eval(("[[10,650], [1023,450], [1023,800], [100,800]]")))
for img in stereo_centre_images:
  wrap_image = four_point_transform(img,pts)
  wrap_image = cv2.resize(wrap_image, (500, 250))
  wrap_stereo_centre.append(wrap_image)

for img, image in enumerate(wrap_stereo_centre):
    plt.figure(figsize=(5,5))
    plt.title(str("Stereo Centre Wrap Perspective Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

###Stitch WarpPerspective Function

A blank image is taken of size (1032,1015,3) on which the mono-left image is pasted on the left, mono-right image is pasted on right side of blank image, stereo centre is pasted on top as the front of the blank image and mono-rear is placed at the bottom on the basis of the shape of each set of images.

The front and rear images were resized above in order to place the images correctly on the blank image.

Mono-Right,Mono-Left and Mono-Rear Images are rotated to paste correctly on the blank image for generating Top-View

An attempt was made to blend the edges to construct the feel of a single image with the help of addWeighted function which is commented below.

The front image was improvised in terms of shade with the help of histogram matching the mono rear image(code mentioned above)

A blank rectangle in the middle represents the car in the parking slot and the overall image generated represents the generated top-view.

In [ ]:
# merge_img =np.zeros((1280,800,3),dtype = np.uint8)
# wrap_mono_left = cv2.resize(rotated_img_left,(600,300))
# print(rotated_img_left.shape)
# wrap_mono_right = cv2.resize(rotated_img_right,(600,300))
# wrap_stereo_centre = cv2.resize(wrap_image,(300,1200))
# print(wrap_image.shape)
# wrap_mono_rear = cv2.resize(rotated_img_rear,(250,1280))

# left_image = (0,0)
# rear_image = (90,980)
# right_image = (550,0)
# front_image = (120,0)

# #blend left and right with front
# #blend_left = cv2.addWeighted(rotated_img_centre,0.5,cv2.resize(rotated_img_left,(600,300)),0.5,0)
# #blend_right = cv2.addWeighted(rotated_img_centre,0.5,cv2.resize(rotated_img_right,(600,300)),0.5,0)

# merge_img =np.zeros((1280,800,3),dtype = np.uint8)
# merge_img[left_image[1]:left_image[1]+wrap_mono_left.shape[0],left_image[0]:left_image[0]+wrap_mono_left.shape[1]]-wrap_mono_left
# merge_img[right_image[1]:right_image[1]+wrap_mono_right.shape[0],right_image[0]:right_image[0]+wrap_mono_right.shape[1]]-wrap_mono_right
# merge_img[front_image[1]:front_image[1]+wrap_stereo_centre.shape[0],front_image[0]:front_image[0]+wrap_stereo_centre.shape[1]]-wrap_stereo_centre
# merge_img[rear_image[1]:rear_image[1]+wrap_mono_rear.shape[0],rear_image[0]:rear_image[0]+wrap_mono_rear.shape[1]]-wrap_mono_rear

# s = cv2.resize(wrap_mono_left,(600,300))
# t = cv2.resize(wrap_mono_right,(600,300))

# merge_img[0:s.shape[0],0:s.shape[1]] = blend_left




In [ ]:
#check sizes of the left,right,rear and centre images to stitch correctly in future code.
print(wrap_mono_right[0].shape)
print(wrap_mono_left[0].shape)
print(wrap_stereo_centre[0].shape)
print(wrap_mono_rear[0].shape)

In [ ]:
stitched_image=[]
for i in range(0,len(wrap_mono_right)):
    black_image =np.zeros((1032,1015,3),dtype = np.uint8)
    x = (1032 - 500) // 2
    y = 0
    black_image[y:y+wrap_stereo_centre[i].shape[0],
                x:x+wrap_stereo_centre[i].shape[1]] = wrap_stereo_centre[i]
    black_image[:, :350, :] = wrap_mono_left[i]
    black_image[:, 665:1015, :] = wrap_mono_right[i]
    black_image[832:1032, :, :] = wrap_mono_rear[i]
    stitched_image.append(black_image)

for img, image in enumerate(stitched_image):
    plt.figure(figsize=(5,5))
    plt.title(str("Stereo Centre Wrap Perspective Images "+ str(img+1)))
    plt.imshow(image)
    plt.axis("off")
    plt.show()

## Task2: Park Slot Detection

### Car Detection

For Object Detection of the cars Faster R-CNN algorithm from torchvision is implemented.Torchvision is a library to build,train and predict neural networks in Tensorflow.

Faster R-CNN is implemented because it consists of only convolution layers resulting in faster training and predicting than other CNN models for object detection.

The model is evaluated by using the standard Mean Average Precision at a threshold of 0.90 to detect cars.

Prediction is done based on the classes set of the cars and remaining objects as background in the image.

In [ ]:
import torch
import torchvision
import numpy as np
import cv2

# Define the device for running the model on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
img_directory = '/content/drive/MyDrive/sample_small/mono_right'
img_files = os.listdir(img_directory)
process_img = []
for img_file in img_files:
  img_path = os.path.join(img_directory,img_file)
  image = cv2.imread(img_path)
  process_img.append(image)
# Set the model to evaluation mode

final_img = []
for i in process_img:
  model.eval()
  # Define the class labels
  classes = ['background', 'car']
  # Define the threshold for classifying a detection as a car
  threshold = 0.97
  image_tensor = torchvision.transforms.functional.to_tensor(i).to(device)
  with torch.no_grad():
    prediction = model([image_tensor])
  # Get the predicted bounding boxes, scores, and labels for the cars
  boxes = prediction[0]['boxes'].cpu().numpy()
  scores = prediction[0]['scores'].cpu().numpy()
  # labels = prediction[0]['labels'].cpu().numpy()
  car_boxes = boxes[scores >= threshold]
  # car_labels = labels[scores >= threshold]
  # Draw the predicted bounding boxes on the image
  for box in car_boxes:
      x1, y1, x2, y2 = box.astype(int)
      rect = cv2.rectangle(i, (x1, y1), (x2, y2), (0, 0, 255), 2)
  final_img.append(i)



In [ ]:
for img, image in enumerate(final_img):
    plt.figure(figsize=(5,5))
    plt.imshow(image)
    plt.title("Car Detection Mono Right Images: "+ str(img+1))
    plt.axis("off")
    plt.show()

### Detect Parking Lines

For Detect Parking Lines following steps were followed which will be explained in detail during the code implementation:

1) Take Images which Detect Cars in the Above Step

2) Apply Gaussian Blur

3) Apply Canny Edge Detection

4) Finding Region of Interest

5) Detection of Parking Lines



####Apply Gaussian Blur

In [ ]:
kernel_size = (15,15)

blur_images = []
# blur all of the images with opencv's gaussian blur function and populate the blur_images list with the results
for i in final_img:
    resultimage = cv2.GaussianBlur(i,kernel_size,0)
    blur_images.append(resultimage)

#display blurred images
for bi, biimage in enumerate(blur_images):
  plt.figure(figsize=(5,5))
  plt.title(str("Gaussian Blur Image of Mono Right Images "))
  plt.imshow(biimage, cmap="gray")
  plt.axis("off")
  plt.show()

#### Apply Canny Edge Detection

In [ ]:
canny_low_threshold = 70
canny_high_threshold = 200

# apply edge detection to the blur_images list of images
canny_images = []
for i in blur_images:
    edge = cv2.Canny(i, canny_low_threshold, canny_high_threshold)
    canny_images.append(edge)
#display edge images
for ci, ciimage in enumerate(canny_images):
  plt.figure(figsize=(5,5))
  plt.title(str("Canny Edge Detection of Mono Right Images"))
  plt.imshow(ciimage, cmap="gray")
  plt.axis("off")
  plt.show()

#### Finding Region of Interest

In [ ]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon defined by "vertices".
    The rest of the image is set to black.
    """
    #define a blank mask
    mask = np.zeros_like(img)

    #define a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (0,255,255) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
def masked_images(raw_images, canny_images, region):
  ysize = raw_images[0].shape[0]
  xsize = raw_images[0].shape[1]
  # define region of interest. The region of interest should include the road area and ignore the background
  # the coordinates will need to be changed, depending on the image set used, as the field of view will be different
  # define the region as a numpy array, of type np.int32

  #region = np.array() # put your code here
  region = np.array([[(0,750),(10,600),(1023,500),(1023,800)]], dtype= np.int32)
  # apply the mask to the Canny edge images
  masked_images = [region_of_interest(img,[region]) for img in canny_images]
  return masked_images



In [ ]:
region = np.array([[(0,750),(10,600),(1023,500),(1023,800)]], dtype= np.int32)

masked_img = masked_images(mono_right_images, canny_images,region)
for img in masked_img:
  plt.figure(figsize=(5,5))
  plt.title(str("Region of Interest of Mono Right Images"))
  plt.imshow(img, cmap="gray")
  plt.axis("off")
  plt.show()

#### Detection of Parking Lines

In [ ]:
# Resize the canny image to match the size of the actual image
detect_parking = []
for i, image in enumerate(masked_img):
  canny_image_resized = cv2.resize(image, (final_img[i].shape[1], final_img[i].shape[0]))

# Convert the canny image to a color image with the same number of channels as the actual image
  canny_image_color = cv2.cvtColor(canny_image_resized, cv2.COLOR_GRAY2RGB)
  canny_image_color[canny_image_resized > 0] = [0, 255, 0]
# Set a blending factor for the canny image (0.5 = 50-50 blend)
  alpha = 0.6
  green = np.array([0, 255, 0], dtype=np.uint8)
# Overlay the canny image on the actual image using alpha blending
  result = cv2.addWeighted(final_img[i], alpha, canny_image_color, 1, 0)
  detect_parking.append(result)

for img in detect_parking:
  plt.figure(figsize=(5,5))
  plt.title(str("Parking Lines Detection of Mono Right Images"))
  plt.imshow(img, cmap="gray")
  plt.axis("off")
  plt.show()

## Parking Slot Segmentation

I tried following the assignments and Jupyter Notebooks provided in the course module but failed to come up with correct implementation of the code.

Possible way to segment the parking slot apart from Neural Network Implementation could be applying Morphological Operations on the image.

FindContours,Draw Contours and then FillPoly function could be applied to segment the parking slot region.

## Result Image

A final output image is created by resizing the Top View and Park Slot Detection and rotated

In [ ]:
def final_image(left,right):
    result_image = []
    final_output = np.zeros((800,1280,3), dtype=np.uint8)

    # Resize the front, right, left, and rear images to specific sizes
    right = cv2.rotate(right, cv2.ROTATE_90_CLOCKWISE)

    left = cv2.resize(left,(640,800))
    right = cv2.resize(right,(640,800))

    # Define the locations where the resized images will be placed in the final merged image
    right_location = (640,0)
    left_location = (0,0)

    # Merge the four images into a single image by placing them in their respective locations
    final_output[left_location[1]:left_location[1]+left.shape[0],left_location[0]:left_location[0]+left.shape[1]]=left
    final_output[right_location[1]:right_location[1]+right.shape[0],right_location[0]:right_location[0]+right.shape[1]]=right
    result_image.append(final_output)

    return final_output

In [ ]:
final_img = final_image(stitched_image[0],detect_parking[0])
plt.figure(figsize=(5,5))
plt.title(str("Resultant Output"))
plt.imshow(final_img)
plt.axis("off")
plt.show()

In [ ]:
final_images1 = []
for i,img in enumerate(stitched_image):
  final_img = final_image(stitched_image[i],detect_parking[i])
  final_images1.append(final_img)

for image in final_images1:
  plt.figure(figsize=(5,5))
  plt.title(str("Resultant Output"))
  plt.imshow(image)
  plt.axis("off")
  plt.show()

##Limitations


1) Parking Slot Segmentation does not work properly.

2) There is potential to improve the Top-View Generated in the code above.

3) Parking Lines not implemented correctly for 2-3 images

4) Even after the front image in Top-View Generation is improvised in terms of quality by histogram matching the image is not a perfect match to left,right and rear images.


##Potential Improvements

1) A different model could have been chosen and tested for better object detection results such as YOLO or SSD(Single Shot Detector).

2) 2 to 3 images do not detect parking lines correctly which can be improvised by using Hough Tranform or Template Matching Technique. Template Matching detects parking lines by matching a template image of a parking line to the corresponding region in the larger image.

3) The Top View generated could be improvised by changing the WarpPerspective points so that cars are not detected and fine road with parking lane is generated.

4) The Top View generated could be improvised by adding addWeighted function code correctly to merge the left with front and rear and right with front and rear correctly.

5) The Parking Slot could be segmented by re-training a neural network model and segmenting the region where parking lines are detected.

6) Top-View Generation could be improved by stitching more number of images than 4 (8/12 etc)are performed in the code above.

7) The quality of Front Image in Top-View Generation could be further improvised by applying images brightness and contrast functions to match with the mono-left,mono-right and mono-rear images.

## References

1) **Detecting Parking Lines:** Embedded Image Processing Assignment 5

2) https://github.com/ori-mrg/robotcar-dataset-sdk/blob/master/python/

3) **Demosaic Function:** Embedded Image Processing Assignment 1

4) **WarpPerspective Function:** https://pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/

5) **UnDistortion Function:** Embedded Image Processing Assignment 3

6) **Improving Colour Shade of Stereo Centre Image:** Intensity transforms, histogram operations.ipynb provided under Embedded Image Processing Laerning Materials-> Jupyter Notebooks on Blackboard.

7) **Car Object Detection:** https://pytorch.org/vision/master/models/faster_rcnn.html